## **BOSTON 2019**

# Extracción

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Definir la URL para Boston 2019 - repositorio público AWS
url = 'https://s3.amazonaws.com/hubway-data/'

# Se solicita la información a la página y usamos BeautifulSoup para obtenre un listado estructurado en XML y extraemos los archivos .zip de 2019
response = requests.get(url)
files_2019 = [
    file.text for file in BeautifulSoup(response.text, 'xml').find_all('Key')
    if file.text.endswith('.zip') and '2019' in file.text
]

print(f"Encontrados {len(files_2019)} archivos para 2019")

Encontrados 12 archivos para 2019


# Transformaciones

### Transformación 1: Filtrado de columnas y transformación 2: Creación campo ciudad

In [3]:
import os
import zipfile

# Definimos el nombre del archivo CSV final y las columnas que se mantendrán en el csv final
output_file = 'Boston_2019.csv'
required_columns = ['starttime', 'stoptime', 'start station latitude', 'start station longitude', 'end station latitude', 'end station longitude', 'usertype']

# Verificamos si el archivo de salida ya existe para no sobreescribirlo
if not os.path.exists(output_file):
    # Crear el archivo CSV con las columnas definidas anteriormente y con la columna adicional 'city'
    pd.DataFrame(columns=required_columns + ['city']).to_csv(output_file, index=False)

# Iteramos sobre cada archivo mensual de 2019
for file_name in files_2019:
    print(f"Procesando archivo: {file_name}")

    # Descargar el archivo y descomprimir
    file_url = url + file_name
    local_zip = file_name.split('/')[-1] #nos quedamos con el nombre del mes del archivo en lugar de toda la dirección URL

    with open(local_zip, "wb") as f: #se crea un archivo vacio para meter la información de la URL de cada mes
        f.write(requests.get(file_url).content)

    # Descomprimir el archivo en una carpeta temporal
    with zipfile.ZipFile(local_zip, "r") as zip_ref: #se abre el archivo ZIP para leerlo
        zip_ref.extractall("temp_data") #se extrae la información y lo ponemos en la carpeta temporal

    # T.1: TRANSFORMACIÓN DE FILTRADO: Cargar el CSV descomprimido y filtrar las columnas necesarias
    csv_file = [file for file in os.listdir("temp_data") if file.endswith('.csv')][0]
    df = pd.read_csv(os.path.join("temp_data", csv_file), usecols=required_columns)

    # T.2: TRANSFORMACIÓN CREACIÓN CAMPO CIUDAD: Añadir la columna 'city' con el valor "Boston" en todas las filas
    df['city'] = 'Boston'

    # Guardar los datos en el archivo CSV final (añadir al archivo existente)
    df.to_csv(output_file, mode='a', index=False, header=False)

    # Limpiar archivos temporales
    os.remove(local_zip)
    os.remove(os.path.join("temp_data", csv_file))

print(f"Archivo '{output_file}' actualizado con todos los meses de 2019 y la columna 'city' añadida.")


Procesando archivo: 201901-bluebikes-tripdata.zip
Procesando archivo: 201902-bluebikes-tripdata.zip
Procesando archivo: 201903-bluebikes-tripdata.zip
Procesando archivo: 201904-bluebikes-tripdata.zip
Procesando archivo: 201905-bluebikes-tripdata.zip
Procesando archivo: 201906-bluebikes-tripdata.zip
Procesando archivo: 201907-bluebikes-tripdata.zip
Procesando archivo: 201908-bluebikes-tripdata.zip
Procesando archivo: 201909-bluebikes-tripdata.zip
Procesando archivo: 201910-bluebikes-tripdata.zip
Procesando archivo: 201911-bluebikes-tripdata.zip
Procesando archivo: 201912-bluebikes-tripdata.zip
Archivo 'Boston_2019.csv' actualizado con todos los meses de 2019 y la columna 'city' añadida.


In [4]:
# Comprobamos que el csv se ha creado correctamente y que las columnas tengan el formato correcto.
df = pd.read_csv('Boston_2019.csv')
df

# Se puede observar que el formato fecha conserva la información de los milisegundos en las últimas filas
# Para nuestro análisis no será necesario tanto nivel de detalle por lo que, será necesario eliminar esta información.

,starttime,stoptime,start station latitude,start station longitude,end station latitude,end station longitude,usertype,city
0,2019-01-01 00:09:13.7980,2019-01-01 00:15:25.3360,42.362131,-71.091156,42.355601,-71.103945,Subscriber,Boston
1,2019-01-01 00:33:56.1820,2019-01-01 00:38:20.8800,42.366162,-71.086883,42.362428,-71.084955,Subscriber,Boston
2,2019-01-01 00:41:54.6000,2019-01-01 00:49:33.2730,42.365070,-71.103100,42.373379,-71.111075,Subscriber,Boston
3,2019-01-01 00:43:32.5710,2019-01-01 00:49:37.4260,42.379011,-71.119945,42.391210,-71.122608,Subscriber,Boston
4,2019-01-01 00:49:56.4640,2019-01-01 01:01:17.7010,42.373231,-71.120886,42.383932,-71.139613,Subscriber,Boston
...,...,...,...,...,...,...,...,...
2522766,2019-12-31 23:52:10.7280,2020-01-01 00:18:13.4180,42.352175,-71.055547,42.352175,-71.055547,Customer,Boston
2522767,2019-12-31 23:52:14.9270,2020-01-01 00:29:45.8210,42.352175,-71.055547,42.352175,-71.055547,Customer,Boston
2522768,2019-12-31 23:54:58.3910,2020-01-01 00:01:35.4790,42.368605,-71.099302,42.361619,-71.080436,Subscriber,Boston
2522769,2019-12-31 23:55:54.5440,2020-01-01 00:01:55.4270,42.362811,-71.056067,42.371848,-71.060292,Subscriber,Boston


### Transformación 4: Corrección formato fecha

In [5]:
# Reemplaza los milisegundos por vacio, solo cuando están presentes (buscando cualquier dato que tenga un punto seguido de números, esto son los milisegundos)
for columna in ['starttime', 'stoptime']:
    df[columna] = df[columna].str.replace(r'\.\d+$', '', regex=True) #busca en las columnas los puntos (\.) seguidos de números (\d) y selecciona el final de la cadena (+$)

# Verificación rápida
df[['starttime', 'stoptime']].tail()

,starttime,stoptime
2522766,2019-12-31 23:52:10,2020-01-01 00:18:13
2522767,2019-12-31 23:52:14,2020-01-01 00:29:45
2522768,2019-12-31 23:54:58,2020-01-01 00:01:35
2522769,2019-12-31 23:55:54,2020-01-01 00:01:55
2522770,2019-12-31 23:57:49,2019-12-31 23:58:52


In [6]:
# Guardar en csv tras las transformaciones pertinentes:
df.to_csv('Boston_2019.csv', index=False)